## Blanc


In [2]:
from blanc import BlancHelp, BlancTune
import time

In [3]:
doc = {
    "document": "Seeing through the Brain: Image Reconstruction of Visual Perception from Human Brain Signals Yu-Ting Lan1*, Kan Ren2, Yansen Wang2, Wei-Long Zheng1, Dongsheng Li2, Bao-Liang Lu1, Lili Qiu2 1Shanghai Jiao Tong University,2Microsoft Research {kanren, yansenwang }@microsoft.com, weilong@sjtu.edu.cn Abstract Seeing is believing , however, the underlying mechanism of how human visual perceptions are intertwined with our cog- nitions is still a mystery. Thanks to the recent advances in both neuroscience and artificial intelligence, we have been able to record the visually evoked brain activities and mimic the visual perception ability through computational approaches. In this paper, we pay attention to visual stimuli reconstruction by reconstructing the observed images based on portably accessible brain signals, i.e., electroencephalog- raphy (EEG) data. Since EEG signals are dynamic in thetime-series format and are notorious to be noisy, processingand extracting useful information requires more dedicated ef-forts; In this paper, we propose a comprehensive pipeline,named N EURO IMAGEN , for reconstructing visual stimuli im-ages from EEG signals. Specifically, we incorporate a novelmulti-level perceptual information decoding to draw multi-grained outputs from the given EEG data. A latent diffu-sion model will then leverage the extracted information toreconstruct the high-resolution visual stimuli images. The ex-perimental results have illustrated the effectiveness of imagereconstruction and superior quantitative performance of ourproposed method.IntroductionUnderstanding cortical responses to human visual percep-tion has emerged a research hotspot, which can significantlymotivate the development of computational cognitive sys-tem with the knowledge of neuroscience (Palazzo et al.2020). Along with the rapid development of physiologicaltechniques such as functional magnetic resonance imaging(fMRI) or electroencephalograph (EEG), it becomes possi-ble to record the visually-evoked human brain activities forfurther analysis. Thus, the research community put the atten-tion onto these complicated brain signal data and try to re-construct the stimuli contents used for evoking human sub-jects in the experiments, for understanding and simulatingthe human visual perception.One of the mainstream attempts to study the human vi-sual perception is to reconstruct the seen contents such asimages (Takagi and Nishimoto 2023) or videos (Chen, Qing,and Zhou 2023) used to evoke the human subjective in the*Work done during Yuting’s intern at Microsoft Research Asia,correspondence to Kan Ren, Yansen Wang and Weilong Zheng.stimuli experiments, via computational approaches such asdeep neural networks. These works are mainly based onfMRI data (Allen et al. 2022), while collecting these imagingdata requires expensive devices and lacks of convenience forpractical usage. In contrast, EEG has provided a more expe-dient solution to record and analyze brain signals, yet fewworks are learning visual perception upon these brain signaldata. EEG data are commonly time-series electrophysiolog-ical signals recorded via electrodes placed upon the humanscalp, while the subjects are watching some stimuli contentssuch as images which have also been temporally aligned tothe recorded signals in the data.Though more convenient, reconstruction of visual stim-uli from EEG signals are more challenging than that fromfMRI data. First, EEG signals are in time-series data for-mat, which is temporal sequence and quite different to thestatic 2D/3D images, leading to the challenge of the match-ing stimuli to the corresponding brain signal pieces. Second,the effects of electrode misplacement or body motion resultin severe artifacts in the data with quite low signal-to-noiseratio (SNR), which have largely influenced the modeling andunderstanding of the brain activities. Simply mapping thein severe artifacts in the data with quite low signal-to-noiseratio (SNR), which have largely influenced the modeling andunderstanding of the brain activities. Simply mapping theEEG input to the pixel domain to recover the visual stim-uli is of low quality. The existing works tackling image re-construction from EEG either traditional generative modelsfrom scratch (Kavasidis et al. 2017) and fine-tuning largegenerative models (Bai et al. 2023), which are less efficient;or just retrieving similar images from the data pool (Ye et al.2022). They fail to capture semantic information or recon-struct high-resolution outputs.In this work, we propose a comprehensive pipeline forNeural Image generation, namely N EURO IMAGEN , fromhuman brain signals. To tackle with aforementioned chal-lenges in this task, we incorporate a multi-level seman-tics extraction module which decodes different semantic in-formation from the input signal with various granularity.Specifically, the extracted information contains sample-levelsemantics which is easy to decode, and pixel-level seman-tics such as the saliency map of silhouette information thattends to more decoding difficulties. The multi-level outputsare further fed into the pretrained diffusion models with thecontrol of the generation semantics. Through this way, ourmethod can flexibly handle the semantic information extrac-tion and decoding problem at different granularity and diffi-arXiv:2308.02510v2  [eess.IV]  16 Aug 2023culties, which can subsequently facilitate the generation viaeffectively controlling the fixed downstream diffusion modelat different levels.We evaluate our methods with the traditional image re-construction solutions on EEG data. The results demon-strate the superiority of our N EURO IMAGEN over the com-pared methods in both quantitative and qualitative results onthe EEG-image dataset. The proposed multi-level semanticsextraction at different granularity can largely increase thestructure similarity and semantic accuracy of reconstructedimages with the observed visual stimuli.Related WorkDiffusion ModelsRecently, diffusion models have emerged as state-of-the-art approaches in the field of generative models for sev-eral tasks, including image synthesis, video generation, andmolecule design (Yang et al. 2022; Song, Meng, and Ermon2020; Dhariwal and Nichol 2021). A denoising diffusionprobabilistic model (DDPM) (Ho, Jain, and Abbeel 2020;Sohl-Dickstein et al. 2015) is a parameterized bi-directionalMarkov chain using variational inference to produce samplematching after a finite time. The forward diffusion processis typically designed with the goal to transform any data dis-tribution into a simple prior distribution ( e.g., an isotropicGaussian), and the reverse denoising diffusion process re-verses the former by learning transition kernels parameter-ized by deep neural networks, such as U-Net (Ronneberger,Fischer, and Brox 2015). However, DDPM operates and un-dergoes evaluation and optimization in pixel space, leadingto slower inference speeds and higher training costs. To ad-dress these limitations, Rombach et al.(2022) introduced theconcept of latent diffusion models (LDMs). In LDMs, diffu-sion models are applied within the latent space of the pow-erful pretrained autoencoders. This approach proves to bean effective generative model, accompanied by a separatecompression stage that selectively eliminates imperceptibledetails. By operating in the latent space, LDMs overcomethe drawbacks of pixel space evaluation, enabling faster in-ference and reducing training costs.Image Decoding from fMRIThe most recent works reconstructing the stimuli contentsfrom the brain activities are mainly focused on fMRI data.fMRI, as the measurement of the brain’s blood-oxygen-level-dependent (BOLD) signals, has seen substantial ad-vancements in brain signal decoding. The conventional vi-sual decoding methods in fMRI usually rely on trainingdeep generative neural networks, such as generative adver-sarial networks (GAN) and variational autoencoders (V AE)with paired brain-image data (Shen et al. 2019; Beliy et al.2019). However, the decoding performance of these conven-tional methods is usually limited, and they struggle to pro-duce visual contents with high resolution, because traininga deep generative model from scratch is in general challeng-ing and the dataset of brain signals is relatively small andnoisy. Thus, recent research attempts to directly map brainsignals into carefully pretrained latent spaces and finetunelarge-scale pretrained models to generate diverse and high-resolution images. Takagi and Nishimoto map the brain ac-tivities to latent space and convert them to natural imagesusing LDM. MinD-Vis (Chen et al. 2023) integrates maskbrain modelings and LDM to generate more plausible im-ages with preserved semantic information. Zeng et al. inte-grate silhouette information from brain signals with a con-trollable diffusion model to reconstruct high-quality imagesconsistent with original visual stimuli. These methods gen-erate more plausible and semantically meaningful images.Image Decoding from EEG SignalsEEG is more portable but has relatively lower spatial res-olution and suffers from larger noise, compared to fMRI,which makes decoding visual experience from brain sig-nals a challenging problem. Brain2Image (Kavasidis et al.olution and suffers from larger noise, compared to fMRI,which makes decoding visual experience from brain sig-nals a challenging problem. Brain2Image (Kavasidis et al.2017) implements long short-term memory (LSTM) stackedwith GAN and V AE techniques to generate seen images ofImageNet (Krizhevsky, Sutskever, and Hinton 2012) fromEEG signals (Kavasidis et al. 2017). Neurovison (Khareet al. 2022) proposes conditional progressive growing ofGAN (CProGAN) to develop perceived images and showeda higher inception score. Ye et al. focuses on cross-modalalignment and retrieves images at the instance level, ensur-ing distinguishable model output for EEG signals. We alsonote that there is a parallel work DreamDiffusion (Bai et al.2023) to ours, which finetunes the diffusion model with anauxiliary task for aligning the EEG data and Image CLIPembeddings. However, the end-to-end training framework ofDreamDiffusion struggles to effectively decode and utilizemulti-level semantic information from EEG signals, whichlimits its ability to handle inherent noise characteristics. Inaddition, DreamDiffusion requires fine-tuning the diffusionmodels, which poses practical challenges and limitations interms of scalability and efficiency.MethodologyIn this section, we design our method, N EURO IMAGEN , toextract multi-level semantics from EEG signals and subse-quently integrate them into a pretrained diffusion model toreconstruct the observed visual stimuli from EEG signals.We briefly introduce the intuition of multi-level seman-tics extraction in our N EURO IMAGEN . EEG signals are non-stationary time-series signals and are easy to disturb by arti-facts like body motion, resulting in the low SNR of the sig-nals. To tackle this challenge, we decode different semanticinformation with various granularity. Specifically, the pixel-level semantics such as the saliency map of silhouette in-formation preserve fine-grained color, position, and shapedetails of the observed stimuli. The sample-level semanticsprovides a coarse-grained description of the visual stimuli,such the concept or category of the visual content. Thesedesigns exhibit the capacity to effectively manage the chal-lenges posed by noisy time-series EEG data, consequentlyfacilitating the reconstruction of high-quality visual stimuli.In the following, we first formulate the problem andgive an overview of N EURO IMAGEN . Then, we describethe multi-level semantics extractions of the N EURO IMA-EEG SignalsVisual StimuliReconstructed ImageMulti -level InformationCLIP Embedding Decoding From EEG SignalsPixel Level Sample Level Fine-grained ControlLatent Diffusion Modelt= 0 Diffusion ProcessDenoising ProcessCoarse-grained Control“An image of airliner”CLIPCaptionCLIP EmbeddingSupervisionSupervisionpTpMsMldmEldmDFigure 1: Overview of our N EURO IMAGEN . All the modules with dotted lines, i.e.pixel-level supervision and sample-levelsupervision, are only used during the training phase. and would be removed during the inference phase .GEN, including pixel-level semantics and sample-level se-mantics with the corresponding training details of the decod-ing procedure. Finally, we detail the image reconstructionprocedure of N EURO IMAGEN , which integrates the coarse-grained and fine-grained semantics with a pretrained latentdiffusion model to reconstruct the observed visual stimulifrom EEG signals.Problem StatementIn this section, we formulate the problem and givean overview of N EURO IMAGEN . Let the paired{(EEG,image )}dataset as Ω = {(xi, yi)}ni=1, whereyi∈ RH×W×3is the visual stimuli image to evoke thebrain activities and xi∈ RC×Trepresents the recordedcorresponding EEG signals. Here, Cis the channel numberof EEG sensors and Tis the temporal length of a sequenceassociated with the observed image. The general objectiveof this research is to reconstruct an image yusing thecorresponding EEG signals x, with a focus on achieving ahigh degree of similarity to the observed visual stimuli.Multi-level Semantics Extraction FrameworkFigure 1 illustrates the architecture of N EURO IMAGEN .In our approach, we extract multi-level semantics, repre-sented as {M1(x), M 2(x),···, Mn(x)}, which capture var-ious granularity ranges from coarse-grained to fine-grainedinformation from EEG signals corresponding to visual stim-uli. The coarse-grained semantics serves as a high-leveloverview, facilitating a quick understanding of primary at-tributes and categories of the visual stimuli. On the otherhand, fine-grained semantics offers more detailed informa-tion, such as localized features, subtle variations, and small-scale patterns. The multi-level semantics are then fed into ahigh-quality image reconstructing module Fto reconstructthe visual stimuli ˆy=F[M1(x), M 2(x),···, Mn(x)].Specifically, we give two-level semantics as follows. Let MpandMsbe the pixel-level semantic extractor and sample-level semantic extractor, respectively. Pixel-level semanticsis defined as the saliency map of silhouette informationMp(x)∈ RHp×Wp×3. This step enables us to analyze theEEG signals in the pixel space and provide the rough struc-ture information. Subsequently, we define the sample-levelsemantics as Ms(x)∈ RL×Ds, to provide coarse-grainedinformation such as image category or text caption.To fully utilize the two-level semantics, the high-qualityimage reconstructing module Fis a latent diffusion model.It begins with the saliency map Mp(x)as the initialimage and utilizes the sample-level semantics Ms(x)topolish the saliency map and finally reconstruct ˆy=F(Mp(x), Ms(x)).Pixel-level Semantics ExtractionIn this section, we describe how we decode the pixel-levelsemantics, i.e.the saliency map of silhouette information.The intuition of this pixel-level semantics extraction is tocapture the color, position, and shape information of the ob-served visual stimuli, which is fine-grained and extremelydifficult to reconstruct from the noisy EEG signal. However,as is shown in Figure 3, despite the low image resolution andlimited semantic accuracy, such a saliency map successfullycaptures the rough structure information of visual stimulifrom the noisy EEG signals. Specifically, our pixel-level se-mantics extractor Mpconsists of two components: (1) con-trastive feature learning to obtain discriminative features ofEEG signals and (2) the estimation of the saliency map ofmantics extractor Mpconsists of two components: (1) con-trastive feature learning to obtain discriminative features ofEEG signals and (2) the estimation of the saliency map ofsilhouette information based on the learned EEG features.Contrastive Feature Learning We use contrastive learn-ing techniques to bring together the embeddings of EEGsignals when people get similar visual stimulus, i.e. see-ing images of the same class. The triplet loss (Schroff,Kalenichenko, and Philbin 2015) is utilized asLtriplet = max(0 , β+∥fθ(xa)−fθ(xp)∥22−∥fθ(xa)−fθ(xn)∥22),(1)where fθis the feature extraction function (Kavasidis et al.2017) that maps EEG signals to a feature space. xa, xp, xnare the sampled anchor, positive, and negative EEG signalsegments, respectively. The objective of eq. (1) is to mini-mize the distance between xaandxpwith the same labels(the class of viewed visual stimuli) while maximizing thedistance between xaandxnwith different labels. To avoidthe compression of data representations into a small clusterby the feature extraction network, a margin term βis incor-porated into the triplet loss.Estimation of Saliency Map After we obtain the featureof EEG signal fθ(x), we can now generate the saliency mapof silhouette information from it and a random sampled la-tentz∼ N(0,1), i.e.,Mp(x) =G(z, fθ(x)).Gdenotes for the saliency map generator. In this paper,we use the generator from the Generative Adversarial Net-work(GAN) (Goodfellow et al. 2020) framework to generatethe saliency map and the adversarial loss is defined as fol-lows:LDadv= max(0 ,1−D(A(y), fθ(x)))+max(0 ,1 +D(A(Mp(x))), fθ(x))), (2)LGadv=−D(A(Mp(x)), fθ(x)). (3)In GAN, besides the generator G, a discriminator Dis in-troduced to distinguish between images from the generatorand ground truth images x. It is optimized by minimizing thehinge loss (Lim and Ye 2017) defined in Equation (2). Aisthe differentiable augmentation function (Zhao et al. 2020).To stabilize the adversarial training process and alleviate theproblem of mode collapse, we add the mode seeking regu-larization (Mao et al. 2019):Lms=−dx(G(z1, fθ(x)), G(z2, fθ(x)))dz(z1, z2), (4)where d∗(·)denotes the distance metric in image space xorlatent space zandz1, z2∼ N (0,1)are two different sam-pled latent vectors.To enforce the accuracy of the generated saliency mapfrom the visual stimuli, we use the observed image as super-vision and incorporate the Structural Similarity Index Mea-sure (SSIM) as well:LSSIM= 1−2µxµMp(x)+C1 2σxσMp(x)+C2µ2x+µ2Mp(x)+C1 σ2x+σ2Mp(x)+C2,(5)where µx,µMp(x),σx, and σMp(x)represent the mean andstandard values of the ground truth images and reconstructedsaliency maps of the generator. C1andC2are constants tostabilize the calculation.The final loss for the generator is the weighted sum of thelosses:LG=α1· LGadv+α2· Lms+α3· LSSIM, (6)andαi∈{1,2,3}are hyperparameters to balance the loss terms.Sample-level Semantics ExtractionAs aforementioned, the EEG signals are notorious for theirinherent noise, making it challenging to extract both pre-cise and fine-grained information. Therefore, besides fine-grained pixel-level semantics, we also involve sample-levelsemantic extraction methods to derive some coarse-grainedinformation such as the category of the main objects of theimage content. These features have a relatively lower rankand are easier to be aligned. Despite being less detailed,these features can still provide accurate coarse-grained in-formation, which is meaningful to reconstruct the observedvisual stimuli.Specifically, the process Mswill try to align the infor-mation decoded from the input EEG signals to some gen-erated image captions, which are generated by some otheradditional annotation model such as Contrastive Language-Image Pretraining (CLIP) model (Radford et al. 2021). Be-low we detail the processes of image caption ground-truthgeneration and semantic decoding with alignment.An image of african elephantAn elephant standing next to a large rockAn image of mountain bikeA man riding a mountain bike down a trail in the woodsAn image ofparachuteA person flying a parachute in the air with a bannerAn image of daisyA red and white flower with yellow centerGT images Label captions BLIP captionsFigure 2: Examples of ground-truth images, label captions,and BLIP captions, respectively.Generation of Image Captions We propose two methodsto generate the caption for each image to help supervise thedecoding procedure of semantic information from EEG sig-nals. Since the observed images are from ImageNet datasetto generate the caption for each image to help supervise thedecoding procedure of semantic information from EEG sig-nals. Since the observed images are from ImageNet datasetcontaining the class of the image, we define a straightfor-ward and heuristic method of label caption, which utilizesthe class name of each image as the caption, as illustrated inthe middle column of Figure 2. The second method is thatwe use an image caption model BLIP (Li et al. 2023), whichis a generic and computation-efficient vision-language pre-training (VLP) model utilizing the pretrained vision modeland large language models. We opt for the default parameterconfiguration of the BLIP model to caption our images. Theexamples are demonstrated in the right column of Figure 2.As can be seen, the label captions tend to focus predomi-nantly on class-level information, and the BLIP-derived cap-tions introduce further details on a per-image level.Predict the Text CLIP Embedding After the generationof the image caption ground-truth, the goal of the semanticdecoding is to extract the information from the EEG signalsto align the caption information. Note that, this procedure isconducted in the latent space, where the latent embeddingshave been processed from the CLIP model from the abovegenerated captions. Specifically, We extracted the CLIP em-beddings ˆhclip∗from the generated captions and align theoutput hclipof EEG sample-level encoder with the loss func-tion asLclip=||hclip−ˆhclip∗||22, (7)where ∗ ∈ { B, L}denotes the BLIP caption embedding orlabel caption embedding.Combining Multi-level EEG Semantics withDiffusion ModelIn this section, we present a comprehensive explanationof how multi-level semantics can be effectively integratedinto a diffusion model for visual stimulus reconstruction.We utilize both pixel-level semantics, denoted as Mp(x)(obtained using G(z, fθ(x))), and sample-level semantics,represented as Ms(x)(obtained using hclip), to exert vari-ous granularity control over the image reconstruction pro-cess. The reconstructed visual stimuli are defined as ˆy=F(Mp(x), Ms(x)) =F(G(fθ(x), hclip)).Specifically, we used the latent diffusion model to per-form image-to-image reconstructing with the guidance ofconditional text prompt embeddings: (1) First, we recon-struct the pixel-level semantics G(fθ(x))from EEG sig-nals and resize it to the resolution of observed visual stimuli(2)G(fθ(x))is then processed by the encoder Eldmof au-toencoder from the latent diffusion model and added noisethrough the diffusion process. (3) Then, we integrate thesample-level semantics hclipas the cross-attention input ofthe U-Net to guide the denoising process. (4) We project theoutput of the denoising process to image space with Dldmand finally reconstruct the high-quality image ˆy.ExperimentsDatasetThe effectiveness of our proposed methodology is validatedusing the EEG-image dataset (Spampinato et al. 2017). Thisdataset is publicly accessible and consists of EEG data gath-ered from six subjects. The data was collected by present-ing visual stimuli to the subjects, incorporating 50 imagesfrom 40 distinct categories within the ImageNet dataset(Krizhevsky, Sutskever, and Hinton 2012). Each set of stim-uli was displayed in 25-second intervals, separated by a 10-second blackout period intended to reset the visual pathway.This process resulted in totally 2000 images, with each ex-periment lasting 1,400 seconds (approximately 23 minutesand 20 seconds). The EEG-image dataset encompasses adiverse range of image classes, including animals (such aspandas), and objects (such as airlines).Following the common data split strategy (Kavasidis et al.2017), we divide the pre-processed raw EEG signals andtheir corresponding images into training, validation, andtesting sets, with corresponding proportions of 80% (1,600images), 10% (200 images), and 10% (200 images) andbuild one model for all subjects. The dataset is split by im-ages, ensuring the EEG signals of all subjects in response toa single image are not spread over splits.Evaluation MetricsN-way Top- kClassification Accuracy (ACC) Following(Chen et al. 2023), we evaluate the semantic correctness ofour reconstructed images by employing the N-way top- kclassification accuracy. Specifically, the ground truth imageyand reconstructed image ˆyare fed into a pretrained Im-ageNet1k classifier (Dosovitskiy et al. 2020), which deter-mines whether yandˆybelong to the same class. Then weyand reconstructed image ˆyare fed into a pretrained Im-ageNet1k classifier (Dosovitskiy et al. 2020), which deter-mines whether yandˆybelong to the same class. Then wecheck for the reconstructed image if the top- kclassificationinNselected classes matches the class of ground-truth im-age. Importantly, this evaluation metric eliminates the needfor pre-defined labels for the images and serves as an indi-cator of the semantic consistency between the ground truthand reconstructed images. In this paper, we select 50-waytop-1 accuracy as the evaluation metric.Inception Score (IS) IS, introduced by (Salimans et al.2016), is commonly employed to evaluate the quality anddiversity of reconstructed images in generative models.To compute the IS, a pretrained Inception-v3 classifier(Szegedy et al. 2016) is utilized to calculate the class prob-abilities for the reconstructed images. We use IS to give aquantitative comparison between our method and baselines.Structural Similarity Index Measure (SSIM) SSIM of-fers a comprehensive and perceptually relevant metric forimage quality evaluation. SSIM is computed over multiplewindows of the ground truth image and the correspondingreconstructed image in luminance, contrast, and structurecomponents, respectively.ResultsExperiment Results on the ImageNet DatasetThe main results are illustrated in Figure 3. The imagespositioned on the left with red boxes represent the groundtruth images. The second images from the left represent thesaliency map reconstructed from EEG signals. The three im-ages on the right exhibit the three sampling results for thegiven pixel-level saliency map with the guidance of sample-level semantics of EEG signals. Upon comparison with theground truth images and the reconstructed saliency maps,we validate that our pixel-level semantics extraction fromEEG signals successfully captures the color, positional, andshape information of viewed images, despite limited seman-tic accuracy. Comparing the GT images and three recon-structed samples, it is demonstrated that the latent diffu-sion model successfully polishes the decoded saliency mapGT images Sample1 Sample2 Sample3 Saliency Map GT images Sample1 Sample2 Sample3 Saliency MapFigure 3: The main results of our N EURO IMAGEN . The images positioned on the left with red boxes represent the ground truthimages. The second images from the left represent the pixel-level saliency map reconstructed from EEG signals. The threeimages on the right exhibit the three sampling results for the given saliency map under the guidance of sample-level semantics.with coarse-grained but accurate guidance of sample-levelsemantics from EEG signals. The high-quality reconstructedimages purely from brain signals are perceptually and se-mantically similar to the viewed images.Model ACC (%) IS SSIMBrain2Image 5.01NeuroVision 5.23NEURO IMAGEN 85.6 33.50 0.249Table 1: The quantitative results of our N EURO IMAGEN ,Brain2Image (Kavasidis et al. 2017) and NeuroVision(Khare et al. 2022) on EEG-image dataset.Comparison with BaselinesThe quantitative results of N EURO IMAGEN and baselinesare listed in Table 1. We have introduced the IS reportedin the relevant literature, to exemplify the quality of thereconstructed images. The IS is calculated by encompass-ing all images reconstructed across all subjects and allclasses within the test set. As is demonstrated in Table 1,the IS of our N EURO IMAGEN is significantly higher thanBrain2Image and NeuroVision. Furthermore, inspired by(Bai et al. 2023), we provide a qualitative comparison withthe baselines in Figure 4. As can be seen, the quality of theimages reconstructed by our N EURO IMAGEN is markedlyhigher than those reconstructed by the Brain2Image. Thisobserved enhancement serves to validate the effectivenessand superiority of our proposed methodology.Subject ACC (%) IS SSIMsubj 01 83.84 32.64 0.254subj 02 84.26 32.33 0.247subj 03 86.66 32.93 0.251subj 04 86.48 32.40 0.244subj 05 87.62 32.97 0.250subj 06 85.25 31.76 0.245Table 2: The quantitative results of different subjects.Generation Consistency in Different SubjectsSince EEG signals are subject-specific cognitive processesthat differ significantly in different subjects. In this section,we validate the robustness and feasibility of N EURO IMA-GEN across different individuals. As is illustrated in Fig-ure 5. The quantitative metric of different subjects are stable,which proves the generalization ability of N EURO IMAGEN .The qualitative results are shown in Figure 5. It can be seenthe sampling from different subjects are semantically similarto the ground truth images.Ablation StudyWe further conduct experiments on the EEG-image datasetto analyze the effectiveness of each module of our N EU-ROIMAGEN . We define BandLas the sample-level seman-tics from EEG signals using BLIP caption as supervision orlabel caption as supervision. We define Ias the pixel seman-tics from EEG signals. The effectiveness of different meth-ods is verified by employing ACC, IS, and SSIM.Brain2Image OursAirliner Panda Jack -o’-LanternFigure 4: Comparison baseline Brain2Image (Kavasidis et al. 2017) and our proposed N EURO IMAGEN in three classes, namely’Airliner’, ’Panda’, and ’Jack-o’-Lantern’. The first and second row depicts the results of Brain2Image and our N EURO IMAGEN ,respectively.Model B L I ACC(%) IS SSIM1%%! 4.5 16.31 0.2342%!% 85.9 34.12 0.1803!%% 74.1 29.87 0.1574!%! 65.3 25.86 0.2355%!! 85.6 33.50 0.249Table 3: Quantitative results of ablation studies. BandLrepresent the semantic decoding using BLIP caption and la-bel caption from EEG signals, respectively. Irepresents theperceptual information decoding from EEG signals.Pixel-level Semantics To demonstrate the effectiveness ofthe pixel-level semantics from EEG signals, we conduct val-idation on models 2, 3, 4, and 5. By comparing 2 with 5 and3 with 4, we find that using the pixel-level semantics, i.e.thesaliency map, can significantly increase the structure simi-larity of the reconstructed images and ground truth images.Sample-level Semantics We investigate the module ofsample-level semantics decoding from EEG signals on guid-ing the denoising process. Models 1, 4, and 5 representthe experimental results only using the saliency, both thesaliency map and sample-level semantics with the supervi-sion of BLIP caption, and both the saliency map and sample-level semantics with the supervision of label caption, respec-tively. By comparing 1 with 4 and 1 with 5, the experimen-tal results demonstrate that the use of sample-level seman-tics significantly increases the semantic accuracy of recon-structed images.BLIP Captions vs Label Captions We also compare thetwo caption supervision methods with models 2 with 3 and4 with 5. The experimental results of the label caption in allmetrics are superior to using BLIP caption. We impute theseresults to that the EEG signals may only capture the class-level information. So the prediction of BLIP latent is inaccu-rate, which decreases the performance of diffusion models.ConclusionIn this paper, we explore to understand the visually-evokedbrain activity. Specifically, We proposed a framework,named N EURO IMAGEN , to reconstruct images of visualperceptions from EEG signals. The N EURO IMAGEN firstGT images Subj 01 Subj 02 Subj 03 Subj 04 Subj 06 Subj 05Anemone fish Pizza Electric guitar Cano eFigure 5: Comparison of reconstructed images on different subjects. The images on the left with red boxes represent the groundtruth images. The other six images represent the reconstructed images of different subjects. The shown classes include fish,pizza, guitar, and canoe.generates multi-level semantic information, i.e., pixel-levelsaliency maps and sample-level textual descriptions fromEEG signals, then use the diffusion model to combine theextracted semantics and obtain the high-resolution images.Both qualitative and quantitative experiments reveals thestrong ability of the N EURO IMAGEN .As a preliminary work in this area, we demonstrate thepossibility of linking human visual perceptions with compli-cated EEG signals. We expect the findings can further moti-vate the field of artificial intelligence, cognitive science, andneuroscience to work together and reveal the mystery of ourbrains to proceed visual perception information.ReferencesAllen, E. J.; St-Yves, G.; Wu, Y .; Breedlove, J. L.; Prince,J. S.; Dowdle, L. T.; Nau, M.; Caron, B.; Pestilli, F.; Charest,I.; et al. 2022. A massive 7T fMRI dataset to bridge cogni-tive neuroscience and artificial intelligence. Nature Neuro-science , 25(1): 116–126.Bai, Y .; Wang, X.; Cao, Y .; Ge, Y .; Yuan, C.; and Shan,Y . 2023. DreamDiffusion: Generating High-Quality Imagesfrom Brain EEG Signals. arXiv preprint arXiv:2306.16934 .Beliy, R.; Gaziv, G.; Hoogi, A.; Strappini, F.; Golan, T.;and Irani, M. 2019. From voxels to pixels and back: Self-supervision in natural-image reconstruction from fMRI. Ad-vances in Neural Information Processing Systems , 32.Chen, Z.; Qing, J.; Xiang, T.; Yue, W. L.; and Zhou, J. H.2023. Seeing beyond the brain: Conditional diffusion modelwith sparse masked modeling for vision decoding. In Pro-ceedings of the IEEE/CVF Conference on Computer Visionand Pattern Recognition , 22710–22720.Chen, Z.; Qing, J.; and Zhou, J. H. 2023. Cinematic Mind-scapes: High-quality Video Reconstruction from Brain Ac-tivity. arXiv preprint arXiv:2305.11675 .Dhariwal, P.; and Nichol, A. 2021. Diffusion models beatgans on image synthesis. Advances in Neural InformationProcessing Systems , 34: 8780–8794.Dosovitskiy, A.; Beyer, L.; Kolesnikov, A.; Weissenborn,D.; Zhai, X.; Unterthiner, T.; Dehghani, M.; Minderer, M.;Heigold, G.; Gelly, S.; et al. 2020. An image is worth 16x16words: Transformers for image recognition at scale. arXivpreprint arXiv:2010.11929 .Goodfellow, I.; Pouget-Abadie, J.; Mirza, M.; Xu, B.;Warde-Farley, D.; Ozair, S.; Courville, A.; and Bengio, Y .2020. Generative adversarial networks. Communications ofthe ACM , 63(11): 139–144.Ho, J.; Jain, A.; and Abbeel, P. 2020. Denoising diffusionprobabilistic models. Advances in Neural Information Pro-cessing Systems , 33: 6840–6851.Kavasidis, I.; Palazzo, S.; Spampinato, C.; Giordano, D.; andShah, M. 2017. Brain2image: Converting brain signals intoimages. In Proceedings of the 25th ACM international con-ference on Multimedia , 1809–1817.Khare, S.; Choubey, R. N.; Amar, L.; and Udutalapalli,V . 2022. NeuroVision: perceived image regeneration us-ing cProGAN. Neural Computing and Applications , 34(8):5979–5991.Krizhevsky, A.; Sutskever, I.; and Hinton, G. E. 2012. Im-agenet classification with deep convolutional neural net-works. Advances in Neural Information Processing Systems ,25.Li, J.; Li, D.; Savarese, S.; and Hoi, S. 2023. Blip-2:Bootstrapping language-image pre-training with frozen im-age encoders and large language models. arXiv preprintarXiv:2301.12597 .Lim, J. H.; and Ye, J. C. 2017. Geometric gan. arXiv preprintarXiv:1705.02894 .Mao, Q.; Lee, H.-Y .; Tseng, H.-Y .; Ma, S.; and Yang, M.-H. 2019. Mode seeking generative adversarial networks fordiverse image synthesis. In Proceedings of the IEEE/CVFConference on Computer Vision and Pattern Recognition ,1429–1437.Palazzo, S.; Spampinato, C.; Kavasidis, I.; Giordano, D.;Schmidt, J.; and Shah, M. 2020. Decoding brain represen-tations by multimodal learning of neural activity and visualfeatures. IEEE Transactions on Pattern Analysis and Ma-chine Intelligence , 43(11): 3833–3849.Radford, A.; Kim, J. W.; Hallacy, C.; Ramesh, A.; Goh, G.;Agarwal, S.; Sastry, G.; Askell, A.; Mishkin, P.; Clark, J.;et al. 2021. Learning transferable visual models from nat-ural language supervision. In International Conference onMachine Learning , 8748–8763.Rombach, R.; Blattmann, A.; Lorenz, D.; Esser, P.; and Om-mer, B. 2022. High-resolution image synthesis with latentdiffusion models. In Proceedings of the IEEE/CVF Confer-ence on Computer Vision and Pattern Recognition , 10684–10695.Ronneberger, O.; Fischer, P.; and Brox, T. 2015. U-net: Con-volutional networks for biomedical image segmentation. InMedical Image Computing and Computer-Assisted Interven-tion, 234–241.Salimans, T.; Goodfellow, I.; Zaremba, W.; Cheung, V .; Rad-ford, A.; and Chen, X. 2016. Improved techniques for train-ing gans. Advances in Neural Information Processing Sys-tems, 29.Schroff, F.; Kalenichenko, D.; and Philbin, J. 2015. Facenet:A unified embedding for face recognition and clustering. InProceedings of the IEEE/CVF Conference on Computer Vi-sion and Pattern Recognition , 815–823.Shen, G.; Dwivedi, K.; Majima, K.; Horikawa, T.; andKamitani, Y . 2019. End-to-end deep image reconstructionfrom human brain activity. Frontiers in Computational Neu-roscience , 13: 21.Sohl-Dickstein, J.; Weiss, E.; Maheswaranathan, N.; andGanguli, S. 2015. Deep unsupervised learning usingnonequilibrium thermodynamics. In International Confer-ence on Machine Learning , 2256–2265.Song, J.; Meng, C.; and Ermon, S. 2020. Denoising diffusionimplicit models. arXiv preprint arXiv:2010.02502 .Spampinato, C.; Palazzo, S.; Kavasidis, I.; Giordano, D.;Souly, N.; and Shah, M. 2017. Deep learning human mindfor automated visual classification. In Proceedings of theIEEE/CVF Conference on Computer Vision and PatternRecognition , 6809–6817.Szegedy, C.; Vanhoucke, V .; Ioffe, S.; Shlens, J.; and Wo-jna, Z. 2016. Rethinking the inception architecture for com-puter vision. In Proceedings of the IEEE/CVF Conferenceon Computer Vision and Pattern Recognition , 2818–2826.Takagi, Y .; and Nishimoto, S. 2023. High-resolution im-age reconstruction with latent diffusion models from humanbrain activity. In Proceedings of the IEEE/CVF Conferenceon Computer Vision and Pattern Recognition , 14453–14463.Yang, L.; Zhang, Z.; Song, Y .; Hong, S.; Xu, R.; Zhao, Y .;Shao, Y .; Zhang, W.; Cui, B.; and Yang, M.-H. 2022. Dif-fusion models: A comprehensive survey of methods and ap-plications. arXiv preprint arXiv:2209.00796 .Ye, Z.; Yao, L.; Zhang, Y .; and Gustin, S. 2022.See what you see: Self-supervised cross-modal retrievalof visual stimuli from brain activity. arXiv preprintarXiv:2208.03666 .Ye, Z.; Yao, L.; Zhang, Y .; and Gustin, S. 2022.See what you see: Self-supervised cross-modal retrievalof visual stimuli from brain activity. arXiv preprintarXiv:2208.03666 .Zeng, B.; Li, S.; Liu, X.; Gao, S.; Jiang, X.; Tang, X.; Hu,Y .; Liu, J.; and Zhang, B. 2023. Controllable Mind VisualDiffusion Model. arXiv preprint arXiv:2305.10135 .Zhao, S.; Liu, Z.; Lin, J.; Zhu, J.-Y .; and Han, S. 2020.Differentiable augmentation for data-efficient gan training.Advances in Neural Information Processing Systems , 33:7559–7570.",
    "summary": "The articles discuss the use of EEG signals to reconstruct visual stimuli images and propose a new method, N EURO IMAGEN, which uses multi-level semantics extraction and a latent diffusion model for more accurate and efficient image reconstruction. The method is compared to previous approaches and found to be superior. It also discusses the use of saliency maps and image captions to align EEG signals and reconstruct images. The results show promising results in generating high-quality images from brain signals. The articles also explore the potential applications of these methods in fields such as computer vision and neuroscience.",
}

In [4]:
# document = "Jack drove his minivan to the bazaar to purchase milk and honey for his large family."
# summary = "Jack bought milk and honey."

blanc_help = BlancHelp()
blanc_tune = BlancTune(finetune_mask_evenly=False, show_progress_bar=False)

tik = time.time()
bh = blanc_help.eval_once(doc["document"], doc["summary"])
print(time.time() - tik, "sec")
print("blanc-help:", bh)

tik = time.time()
bt = blanc_tune.eval_once(doc["document"], doc["summary"])
print(time.time() - tik, "sec")
print("blanc-tune:", bt)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model t

99.38256216049194 sec
blanc-help: 0.08355367530407191


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/Users/ms/anaconda3/envs/wisdom-mlab/lib/python3.9/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warn

124.88957500457764 sec
blanc-tune: 0.07006874669487044


## ESTIME


In [5]:
from blanc import Estime

In [10]:
# estimator = Estime()
estimator = Estime(output=["alarms", "alarms_adjusted", "soft", "coherence"])
# text = """In Kander’s telling, Mandel called him up out of the blue a decade or so ago to pitch a project. It made sense why. The two men had similar profiles: Jewish combat veterans in their early 30s. New statewide officeholders in the Midwest."""
# summary = """Kander and Mandel had similar profiles, and it makes sense."""
# estimator.evaluate_claims(text, [summary])
estimator.evaluate_claims(doc["document"], [doc["summary"]])

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[[52, 60.666666666666664, 0.5885415483088721, 0.013589765124156659]]

## Shannon Score


In [11]:
from blanc import Shannon

In [18]:
judge = Shannon(verbose=True)
text = """In Kander’s telling, Mandel called him up out of the blue a decade or so ago to pitch a project. It made sense why. The two men had similar profiles: Jewish combat veterans in their early 30s. New statewide officeholders in the Midwest."""
summary = """Kander and Mandel had similar profiles, and it makes sense."""

# judge.go(text, summary)
judge.go(doc["document"], doc["summary"])
print("!")

Seeing,14.199132479454772
 through,7.606721890856933
 the,1.4783003669012642
 Brain,18.453513146906925
:,6.3168190343144
 Image,9.86285061635545
 Reconstruction,8.680835628880759
 of,2.7021690938885667
 Visual,7.7050695490754295
 Perception,6.3782363998740506
 from,6.007425163817328
 Human,6.932644751508569
 Brain,3.689454197252132
 Sign,8.881527152397755
als,1.835494286131358
 Yu,18.62295764482505
-,3.1809095717170854
T,6.57659321021171
ing,2.0117899875892706
 Lan,12.45765342543317
1,13.379798971641268
*,,13.938374018026936
 Kan,9.699220704315428
 Ren,13.829446214157315
2,2.080924372679408
,,1.8114073815847638
 Y,5.809047600505651
ansen,15.300676219273761
 Wang,9.250381157706
2,5.738741490373541
,,0.54988282156723
 Wei,6.733599895766283
-,3.667156220970775
Long,9.20827112547437
 Zheng,7.0691373517341
1,6.210185600947235
,,0.4312305431960332
 Dong,8.37135221983384
she,8.677064713608937
ng,0.012488153580447919
 Li,3.840058313208589
2,1.664950728816586
,,0.3997234089489314
 B,8.132292361

## G-Eval


### Prompt


In [1]:
# Evaluation prompt template based on G-Eval
EVALUATION_PROMPT_TEMPLATE = """
You will be given one summary written for an article. Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions very carefully. 
Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

{criteria}

Evaluation Steps:

{steps}

Example:

Source Text:

{document}

Summary:

{summary}

Evaluation Form (scores ONLY):

- {metric_name}
"""

# Metric 1: Relevance

RELEVANCY_SCORE_CRITERIA = """
Relevance(1-5) - selection of important content from the source. \
The summary should include only important information from the source document. \
Annotators were instructed to penalize summaries which contained redundancies and excess information.
"""

RELEVANCY_SCORE_STEPS = """
1. Read the summary and the source document carefully.
2. Compare the summary to the source document and identify the main points of the article.
3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
4. Assign a relevance score from 1 to 5.
"""

# Metric 2: Coherence

COHERENCE_SCORE_CRITERIA = """
Coherence(1-5) - the collective quality of all sentences. \
We align this dimension with the DUC quality question of structure and coherence \
whereby "the summary should be well-structured and well-organized. \
The summary should not just be a heap of related information, but should build from sentence to a\
coherent body of information about a topic."
"""

COHERENCE_SCORE_STEPS = """
1. Read the article carefully and identify the main topic and key points.
2. Read the summary and compare it to the article. Check if the summary covers the main topic and key points of the article,
and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.
"""

# Metric 3: Consistency

CONSISTENCY_SCORE_CRITERIA = """
Consistency(1-5) - the factual alignment between the summary and the summarized source. \
A factually consistent summary contains only statements that are entailed by the source document. \
Annotators were also asked to penalize summaries that contained hallucinated facts.
"""

CONSISTENCY_SCORE_STEPS = """
1. Read the article carefully and identify the main facts and details it presents.
2. Read the summary and compare it to the article. Check if the summary contains any factual errors that are not supported by the article.
3. Assign a score for consistency based on the Evaluation Criteria.
"""

# Metric 4: Fluency

FLUENCY_SCORE_CRITERIA = """
Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.
1: Poor. The summary has many errors that make it hard to understand or sound unnatural.
2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
3: Good. The summary has few or no errors and is easy to read and follow.
"""

FLUENCY_SCORE_STEPS = """
Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
"""

### Evaluation code


In [ ]:
def get_geval_score(
    criteria: str, steps: str, document: str, summary: str, metric_name: str
):
    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        criteria=criteria,
        steps=steps,
        metric_name=metric_name,
        document=document,
        summary=summary,
    )
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].message.content


evaluation_metrics = {
    "Relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
    "Coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
    "Consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
    "Fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS),
}

summaries = {"Summary 1": eval_summary_1, "Summary 2": eval_summary_2}

data = {"Evaluation Type": [], "Summary Type": [], "Score": []}

for eval_type, (criteria, steps) in evaluation_metrics.items():
    for summ_type, summary in summaries.items():
        data["Evaluation Type"].append(eval_type)
        data["Summary Type"].append(summ_type)
        result = get_geval_score(criteria, steps, excerpt, summary, eval_type)
        score_num = int(result.strip())
        data["Score"].append(score_num)

pivot_df = pd.DataFrame(data, index=None).pivot(
    index="Evaluation Type", columns="Summary Type", values="Score"
)
styled_pivot_df = pivot_df.style.apply(highlight_max, axis=1)
display(styled_pivot_df)

# Note


- NO REFERENCE SUMMARY!
- ROUGE, BLEU, BERTScore -> can't use
  - can be used to train model on other task (e.g. news summarization)
    - but text length different (too short)
    - Does it matter?
    - What dataset are there to use?
  - maybe use scisummnet? (around 150 words)
- GPT based evaluation (G-Eval paper)
  - Relevance, Coherence, Consistency, Fluency
  - prompt engineering
  - chain of thought


# Reference


- https://cookbook.openai.com/examples/evaluation/how_to_eval_abstractive_summarization (openai cookbook on evaluating text summarization)
- https://www.promptingguide.ai/techniques/cot (chain of thought prompting)
